<a href="https://colab.research.google.com/github/jklee96/MNIST_GAN/blob/main/MNIST_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. 실습 준비: 라이브러리 불러오기**

MNIST 데이터를 사용하여 생성적 적대 신경망(Generative Adversarial Network, GAN)을 구현하고 학습시키는 예제입니다.

먼저, 모델 구현에 필요한 PyTorch 관련 라이브러리와 데이터 시각화를 위한 matplotlib, numpy 등을 불러옵니다.

In [1]:
# Pytorch 관련 기본라이브러리
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, ConcatDataset
from PIL import Image

# torchvision 라이브러리 (데이터셋 및 변환 기능 제공)
import torchvision
import torchvision.transforms as transforms

# 시각화 및 계산을 위한 라이브러리
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil

**2. 학습 환경 및 하이퍼파라미터 설정**

본격적인 모델 구현에 앞서, 학습을 위한 기본 환경과 하이퍼파라미터(Hyperparameter)를 설정합니다.

먼저, torch.cuda.is_available() 코드를 통해 GPU 사용 가능 여부를 확인하고, 사용 가능한 경우 연산 장치를 cuda(GPU)로 설정합니다.

이후 GAN 모델 학습에 필요한 주요 하이퍼파라미터를 정의합니다. 하이퍼파라미터는 모델이 스스로 학습하는 값이 아니라, 개발자가 더 나은 학습 결과를 위해 직접 설정하는 값들을 의미합니다.

**<주요 구성 요소>**

latent_dim: 생성자(Generator)가 이미지를 만들 때 사용할 무작위 노이즈 벡터의 크기입니다. 이 노이즈가 이미지의 '재료'가 됩니다.

epochs: 전체 데이터셋을 몇 번 반복하여 학습할지를 결정합니다.

learning_rate: 학습률은 모델이 정답을 찾아가는 과정에서 가중치(weight)를 얼마나 크게 조정할지를 나타내는 값입니다.

output_dir: 학습 과정에서 생성된 이미지를 저장할 폴더의 이름입니다.

batch_size: 한 번에 학습할 데이터의 묶음(batch) 크기를 의미합니다.

In [2]:
# GPU 사용 가능 여부 확인 후 device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# GAN을 위한 하이퍼파라미터
latent_dim = 100 # Generator의 랜덤 노이즈 입력 차원
epochs = 10 # 학습 에포크 수
learning_rate = 0.0002 # Generator와 Discriminator 모두에 대한 학습률
output_dir = './GAN_output' # 생성된 이미지를 저장할 폴더
batch_size = 64

Using device: cuda


**3. 데이터 준비: MNIST 데이터셋 불러오기**

모델을 학습시키려면 '데이터'가 필요합니다. 이번 실습에서는 손으로 쓴 숫자 이미지로 구성된 MNIST 데이터셋을 사용합니다.

컴퓨터가 이미지를 바로 이해할 수는 없기 때문에, 모델이 학습하기 좋은 형태로 데이터를 가공(전처리)하는 과정이 필요합니다.


transforms.ToTensor(): 이미지를 PyTorch가 다룰 수 있는 텐서(Tensor) 자료형으로 변환합니다.

transforms.Normalize(): 이미지 픽셀 값을 -1에서 1 사이의 값으로 정규화합니다. 이는 모델의 출력 범위와 맞춰주고, 학습을 더 안정적으로 만듭니다.


그 다음, torchvision을 이용해 MNIST 훈련(train) 데이터와 테스트(test) 데이터를 모두 다운로드하고, ConcatDataset으로 두 데이터를 하나로 합쳐 더 많은 양의 데이터를 학습에 사용합니다.

마지막으로, 준비된 전체 데이터셋을 DataLoader에 전달합니다. DataLoader는 데이터를 우리가 설정한 batch_size(64)만큼 작은 묶음으로 알아서 나눠주고, 학습 때마다 데이터의 순서를 무작위로 섞어 모델이 데이터의 순서까지 외우는 것을 방지하는 등 데이터를 효율적으로 공급하는 역할을 합니다.

In [3]:
#------------------------------데이터로더 준비------------------------------#
# MNIST 훈련 데이터셋 및 테스트 데이터셋 다운로드
transform = transforms.Compose([
    transforms.ToTensor(),
    # Tanh를 사용하므로 픽셀 값을 [0, 1]에서 [-1, 1]로 정규화합니다.
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# 훈련 데이터셋과 테스트 데이터셋을 하나로 합치기
combined_dataset = ConcatDataset([train_dataset, test_dataset])

# DataLoader 객체 생성 (배치 단위로 데이터를 로드)
train_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)
# test_loader는 더 이상 사용되지 않으므로 제거하거나 주석 처리합니다.
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 11.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 343kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.19MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.54MB/s]


**4. 모델 구현 (1): 생성자 (Generator) 만들기**

GAN은 생성자(Generator)와 판별자(Discriminator)라는 두 개의 모델이 서로 경쟁하며 학습하는 구조입니다.

이번 셀에서는 생성자 모델을 만듭니다. 생성자의 역할은 '가짜' 이미지를 만드는 '위조지폐범'과 같습니다. 아무것도 없는 상태에서 진짜 같은 가짜 숫자 이미지를 만들어내는 것이 목표입니다.

<생성 과정>

생성자는 latent_dim(100) 차원의 무작위 노이즈(noise) 벡터를 입력으로 받습니다. 이 노이즈는 어떤 이미지를 만들지에 대한 '재료'가 됩니다.

이 작은 노이즈 덩어리를 ConvTranspose2d 라는 layer를 여러 번 통과시키면서, 마치 점토를 펴서 넓히듯이 이미지의 크기를 점차 키워나갑니다.

(100x1x1 노이즈) → (256x7x7) → (128x14x14) → (64x28x28) → (1x28x28 최종 이미지)

최종적으로, Tanh 활성화 함수를 통해 이미지 픽셀 값을 -1에서 1 사이로 조정합니다. 이는 우리가 앞에서 실제 MNIST 데이터의 픽셀 값을 변환했던 범위와 똑같이 맞춰주기 위함입니다.

**<주요 구성 요소>**

ConvTranspose2d: 이미지의 크기(해상도)를 키우는 업샘플링(upsampling) 역할을 하는 핵심 층입니다.

BatchNorm2d: 각 층에서 데이터의 분포를 정규화하여 학습을 안정화시키는 역할을 합니다.

ReLU: 모델이 더 복잡하고 다양한 패턴을 학습할 수 있도록 돕는 대표적인 활성화 함수입니다.

Tanh: 결과물의 픽셀 값을 -1에서 1 사이로 만들어주는 활성화 함수입니다.

In [4]:
#------------------------------모델 준비------------------------------#
# Generator의 입력으로 사용할 랜덤 노이즈 생성 함수
def generate_noise(batch_size, latent_dim, device):
    return torch.randn(batch_size, latent_dim, 1, 1).to(device)

# Generator 클래스 정의: ConvTranspose2d로 업샘플링하여 노이즈를 28x28 이미지로 변환
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # 입력: 잠재 벡터 (latent_dim x 1 x 1)
            nn.ConvTranspose2d(latent_dim, 256, kernel_size=7, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            # 출력: 256 x 7 x 7

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # 출력: 128 x 14 x 14

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # 출력: 64 x 28 x 28

            # 출력 채널을 1로 만들고, 픽셀 값을 [-1, 1] 범위로 조정
            nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1, bias=False),
            nn.Tanh() # Tanh 활성화 함수 사용
        )

    def forward(self, x):
        return self.net(x)

**5. 모델 구현 (2): 판별자 (Discriminator) 만들기**

생성자(Generator)가 '위조지폐범'이라면, 판별자(Discriminator)는 위조지폐를 감별하는 '경찰' 또는 '예술품 감정사'와 같습니다.

판별자의 역할은 이미지를 입력받아, 그 이미지가 실제 데이터(real)인지 아니면 생성자가 만들어낸 가짜 데이터(fake)인지를 구별해내는 것입니다. 이 판별 결과를 생성자에게 알려주면, 생성자는 판별자를 더 잘 속이기 위해 이미지를 개선해 나갑니다.

판별 과정
판별자는 생성자와 정반대의 구조를 가집니다. 28x28 크기의 이미지를 입력받아, Conv2d 층을 통과시키며 이미지에서 특징(feature)을 추출하고 크기를 점차 줄여나갑니다.

(1x28x28 이미지) → (64x14x14) → (128x7x7) → ... → (최종 판별 결과)

여러 층을 거쳐 압축된 특징 정보를 바탕으로, 최종적으로 Sigmoid 활성화 함수를 통해 0과 1 사이의 숫자 하나를 출력합니다.

이 숫자가 1에 가까우면 '진짜' 이미지라고, 0에 가까우면 '가짜' 이미지라고 판단하는 것입니다.

**<주요 구성 요소>**

Conv2d: 이미지의 특징을 추출하고 크기를 줄이는 다운샘플링(downsampling) 역할을 합니다.

LeakyReLU: ReLU와 비슷하지만, GAN의 학습 안정성을 높여주는 것으로 알려진 활성화 함수입니다.

Sigmoid: 결과값을 0과 1 사이의 확률로 만들어, 모델이 '진짜' 또는 '가짜'라는 최종 판별을 내릴 수 있게 합니다.

In [5]:
# Discriminator 클래스 정의: Conv2d로 다운샘플링하여 28x28 이미지를 실제/가짜로 분류
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            # 입력: 1 x 28 x 28 이미지
            nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # 출력: 64 x 14 x 14

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # 출력: 128 x 7 x 7

            nn.Conv2d(128, 256, kernel_size=7, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            # 출력: 256 x 1 x 1

            # 최종적으로 실제/가짜를 판별하는 단일 값 출력
            nn.Conv2d(256, 1, kernel_size=1, stride=1, padding=0, bias=False)
        )

    def forward(self, x):
        # 출력 모양을 [batch_size, 1]로 맞추기 위해 view 사용
        return self.net(x).view(-1, 1)

**6. 모델 생성 및 GPU 할당**

이제 앞서 설계한 Generator와 Discriminator 클래스를 이용해 실제 모델을 만듭니다.

클래스가 '붕어빵 틀'이라면, 이 코드는 틀을 사용해 실제 '붕어빵'(모델 객체)을 찍어내는 과정과 같습니다.

이렇게 생성된 generator와 discriminator 모델은 .to(device) 명령을 통해 GPU에서 계산을 수행하도록 할당됩니다. 복잡한 연산을 GPU가 전담하게

 만들어 학습 속도를 크게 높여주는 과정입니다.

In [6]:
# Generator와 Discriminator 모델을 인스턴스화
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

**7. 손실 함수와 옵티마이저 설정**


모델을 학습시키려면, 모델이 얼마나 잘하고 있는지를 평가하는 '평가 기준(손실 함수)'과, 그 평가를 바탕으로 모델을 개선하는 '개선 방법(옵티마이저)'이 필요합니다.

손실 함수 (Loss Function): '정답'과의 거리 측정하기

손실 함수는 모델의 예측이 정답과 얼마나 다른지를 나타내는 '오차' 또는 '손실(loss)' 값을 계산합니다. 모델은 이 손실 값을 최소화하는 방향으로 학습을 진행합니다.

nn.BCEWithLogitsLoss: GAN은 이미지를 '진짜(1)' 또는 '가짜(0)'로 판별하는 이진 분류 문제이므로, 이진 교차 엔트로피(Binary Cross Entropy, BCE) 손실 함수를 사용합니다. BCEWithLogitsLoss는 마지막 단계의 Sigmoid 함수를 내장하여 계산 과정의 수치적 안정성을 높여주는 역할을 합니다.

옵티마이저 (Optimizer): 똑똑하게 정답 찾아가기

옵티마이저는 손실 함수가 계산한 '오차' 정보를 바탕으로, 모델의 파라미터(가중치)를 더 나은 방향으로 업데이트하는 역할을 합니다. 손실이라는 언덕을 가장 빠르고 효율적으로 내려가는 방법을 찾는 등산가와 같습니다.

torch.optim.Adam: Adam은 현재 가장 널리 사용되는 효율적인 옵티마이저 중 하나입니다.

두 개의 옵티마이저: 생성자와 판별자는 서로 경쟁하며 각자의 목표를 향해 학습하는 별개의 모델입니다. 따라서 각각의 파라미터를 업데이트하기 위한 generator_optimizer와 discriminator_optimizer를 따로 설정해줍니다.

In [7]:
# 손실 함수 정의
# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss() # 수정된 코드

# Optimizer 설정
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate,betas=(0.5, 0.999))

**8. 학습 결과 저장을 위한 준비**

본격적인 학습에 앞서, 생성자가 만들어내는 이미지를 저장하기 위한 마지막 준비를 합니다.

먼저, 이전 학습 결과가 남아있지 않도록 GAN_output 폴더를 깨끗하게 비웁니다. 그 다음, 학습 중에 생성된 이미지를 PNG 파일로 변환하여 저장해주는 save_generated_images 함수를 정의합니다.

이 과정을 통해, 우리는 매 학습 단계마다 생성된 이미지들을 직접 눈으로 확인하며 모델의 성능이 향상되는 과정을 추적할 수 있습니다.

In [8]:
#------------------------------학습 준비------------------------------#
# 학습 전에 GAN_output 폴더 비우기 (기존에 생성된 이미지 삭제)
if os.path.exists(output_dir):
   shutil.rmtree(output_dir)
os.makedirs(output_dir)

# 각 에포크 후에 생성된 이미지를 저장하는 함수 (기존 함수 그대로 사용)
def save_generated_images(images, epoch, output_dir='./GAN_output'):
    os.makedirs(output_dir, exist_ok=True)

    # 이미지를 numpy 배열로 변환하고 [0, 255] 범위로 스케일 조정
    # MNIST 이미지는 이미 [0, 1] 범위이므로 255를 곱하고 uint8로 변환
    images = images.detach().cpu().numpy().squeeze(1)
    images = (images * 255).astype(np.uint8)

    for i in range(images.shape[0]):
        img = Image.fromarray(images[i], mode='L')  # 그레이스케일 이미지
        img.save(os.path.join(output_dir, f'generated_image_epoch{epoch}_{i}.png'))



**9. 모델 학습: 생성자와 판별자의 경쟁**

생성자(위조지폐범)와 판별자(경찰)의 본격적인 경쟁을 통해 서로를 성장시키는 학습 과정을 시작하겠습니다

이 전체 학습은 epochs 수만큼 반복되며, 각 단계는 다음과 같이 두 부분으로 나뉩니다.

**1. 판별자의 훈련 시간**

먼저 판별자를 훈련시킵니다. 판별자의 목표는 '진짜'는 '진짜'로, '가짜'는 '가짜'로 완벽하게 구별해내는 능력을 키우는 것입니다.

진짜 이미지 학습: 실제 MNIST 이미지를 보여주고 "이건 진짜(레이블=1)야"라고 알려줍니다. 판별자는 이 이미지를 보고 1에 가까운 값을 예측하도록 학습합니다.

가짜 이미지 학습: 생성자가 만든 가짜 이미지를 보여주고 "이건 가짜(레이블=0)야"라고 알려줍니다. 판별자는 이 이미지를 보고 0에 가까운 값을 예측하도록 학습합니다.

능력치 업데이트: 위 두 과정에서 발생한 총 오차(loss)를 바탕으로, 옵티마이저가 판별자의 감식안을 더 날카롭게 업데이트합니다.

**2. 생성자의 훈련 시간**

판별자의 훈련이 끝나면, 이번에는 생성자를 훈련시킬 차례입니다. 생성자의 유일한 목표는 판별자를 완벽하게 속여서, 자신이 만든 '가짜' 이미지를 판별자가 '진짜'라고 착각하게 만드는 것입니다.

판별자 속이기: 생성자는 자신이 만든 가짜 이미지를 다시 판별자에게 보여줍니다.

'진짜'라고 우기기: 그리고 판별자가 그 이미지를 보고 '진짜(레이블=1)*라고 판단하도록 속입니다. 판별자가 진짜라고 착각할수록 생성자의 손실(loss)은 낮아집니다.

위조 기술 업데이트: 판별자가 얼마나 속지 않았는지를 나타내는 오차를 바탕으로, 옵티마이저가 생성자의 이미지 생성 기술을 더 정교하게 업데이트합니다.

이 두 과정이 수많은 에포크(epoch) 동안 빠르게 반복됩니다. 훈련이 끝난 후에는 매 에포크마다 저장된 이미지들을 확인하며, 생성자가 점차 진짜 같은 숫자 이미지를 만들어내는 놀라운 과정을 관찰할 수 있습니다!

In [9]:
#------------------------------GAN 학습------------------------------#
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader): # DataLoader에서 이미지와 레이블을 가져옴
        real_images = images.to(device)
        batch_size = real_images.size(0)

        # 실제 레이블(1)과 가짜 레이블(0) 생성 및 [batch_size, 1]로 reshape
        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)

        ### Discriminator 학습 ###
        # 실제 이미지에 대한 순전파
        outputs = discriminator(real_images)
        d_loss_real = criterion(outputs, real_labels) # 실제 이미지는 실제 레이블과 일치해야 함

        # 가짜 이미지 생성
        noise = generate_noise(batch_size, latent_dim, device=device)
        fake_images = generator(noise)

        # 가짜 이미지에 대한 순전파
        outputs = discriminator(fake_images.detach()) # detach로 Generator의 그라디언트가 업데이트되지 않도록 함
        d_loss_fake = criterion(outputs, fake_labels) # 가짜 이미지는 가짜 레이블과 일치해야 함

        # Discriminator에 대한 역전파 및 최적화
        d_loss = d_loss_real + d_loss_fake
        discriminator_optimizer.zero_grad()
        d_loss.backward()
        discriminator_optimizer.step()

        ### Generator 학습 ###
        # 가짜 이미지 생성 및 Discriminator의 예측 가져오기
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels) # Generator는 가짜 이미지를 실제로 분류되도록 학습해야 함

        # Generator에 대한 역전파 및 최적화
        generator_optimizer.zero_grad()
        g_loss.backward()
        generator_optimizer.step()

        # 일정 스텝마다 손실 출력
        if (i+1) % 100 == 0:
             print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')


    # 각 에포크 후에 생성된 이미지를 저장
    with torch.no_grad():
        test_noise = generate_noise(10, latent_dim, device=device) # 테스트용 이미지 10개 생성
        generated_images = generator(test_noise)
        save_generated_images(generated_images, epoch)

    print(f'Epoch [{epoch+1}/{epochs}] completed. Generated images saved.')

Epoch [1/10], Step [100/1094], D_loss: 1.1223, G_loss: 0.9216
Epoch [1/10], Step [200/1094], D_loss: 1.1320, G_loss: 0.9336
Epoch [1/10], Step [300/1094], D_loss: 1.0637, G_loss: 0.9307
Epoch [1/10], Step [400/1094], D_loss: 1.1443, G_loss: 0.9685
Epoch [1/10], Step [500/1094], D_loss: 1.1439, G_loss: 0.9401
Epoch [1/10], Step [600/1094], D_loss: 1.2803, G_loss: 0.9657
Epoch [1/10], Step [700/1094], D_loss: 1.2285, G_loss: 0.9180
Epoch [1/10], Step [800/1094], D_loss: 1.1335, G_loss: 0.9406
Epoch [1/10], Step [900/1094], D_loss: 1.1710, G_loss: 0.9368
Epoch [1/10], Step [1000/1094], D_loss: 1.2073, G_loss: 0.9163


/tmp/ipython-input-2143205352.py:17: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(images[i], mode='L')  # 그레이스케일 이미지


Epoch [1/10] completed. Generated images saved.
Epoch [2/10], Step [100/1094], D_loss: 1.1735, G_loss: 1.0331
Epoch [2/10], Step [200/1094], D_loss: 1.0647, G_loss: 1.0667
Epoch [2/10], Step [300/1094], D_loss: 1.1046, G_loss: 1.0425
Epoch [2/10], Step [400/1094], D_loss: 1.1900, G_loss: 0.9655
Epoch [2/10], Step [500/1094], D_loss: 1.1454, G_loss: 0.9718
Epoch [2/10], Step [600/1094], D_loss: 1.1699, G_loss: 0.9473
Epoch [2/10], Step [700/1094], D_loss: 1.1683, G_loss: 1.0196
Epoch [2/10], Step [800/1094], D_loss: 1.0880, G_loss: 1.0377
Epoch [2/10], Step [900/1094], D_loss: 1.1694, G_loss: 1.0436
Epoch [2/10], Step [1000/1094], D_loss: 1.1215, G_loss: 0.9732
Epoch [2/10] completed. Generated images saved.
Epoch [3/10], Step [100/1094], D_loss: 1.1557, G_loss: 0.9673
Epoch [3/10], Step [200/1094], D_loss: 1.0969, G_loss: 0.9934
Epoch [3/10], Step [300/1094], D_loss: 1.1634, G_loss: 0.9363
Epoch [3/10], Step [400/1094], D_loss: 1.2121, G_loss: 0.9324
Epoch [3/10], Step [500/1094], D_lo